In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.5 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 60.3 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 6.8 MB 72.4 MB/s 
     |████████████████████████████████| 596 kB 80.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 63.2 MB/s 
     |████████████████████████████████| 133 kB 92.7 MB/s 
     |████████████████████████████████| 243 kB 72.7 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 59.8 MB/s 
     |████████████████████████████████| 271 kB 60.2 MB/s 


In [ ]:
from transformers import AdamW, RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from datasets import Dataset, load_dataset
import numpy as np
import pandas as pd
import tqdm
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
NUM_EPOCHS = 3
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 5e-05

In [ ]:
df = pd.read_json('/content/drive/MyDrive/RapMachine/RapNotRap.json')
train = df.sample(frac=0.8) # train split 80%
test = df.drop(train.index) # test split 20%
training_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(test)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenization(batch):
  return tokenizer(batch['text'], padding = True, truncation=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
train_data = training_dataset.map(tokenization, batched = True, batch_size = len(training_dataset))
test_data = validation_dataset.map(tokenization, batched = True, batch_size = len(validation_dataset))

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_set = TensorDataset(train_data['input_ids'], train_data['attention_mask'], train_data['label'])
train_dataloader = DataLoader(train_set, shuffle=True, batch_size=TRAIN_BATCH_SIZE)

test_set = TensorDataset(test_data['input_ids'], test_data['attention_mask'], test_data['label'])
test_dataloader = DataLoader(test_set, shuffle=True, batch_size=EVAL_BATCH_SIZE)

In [ ]:
class RoBERTaBinaryClassifier(nn.Module):
    def __init__(self):
        super(RoBERTaBinaryClassifier, self).__init__()

        self.roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2, return_dict=False)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask=None, labels=None):
        _, pooled_output = self.roberta(input_ids,
                                        attention_mask=attention_mask,
                                        labels=labels,
                                        return_dict=False
                                        )
        
        dropout_output = self.dropout(pooled_output)
        logits = self.linear(dropout_output)
        proba = self.sigmoid(logits)
        return proba

In [ ]:
model = RoBERTaBinaryClassifier()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(NUM_EPOCHS):
    # setup loop with TQDM and dataloader
    loop = tqdm.tqdm(train_dataloader)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optimizer.zero_grad()
        # pull all tensor batches required for training
        input_ids, attention_mask, labels = (tensor.to(device) for tensor in batch)
        # process
        logits = model(input_ids, attention_mask=attention_mask, labels=labels)
        # extract loss
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels.unsqueeze(1).float())
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optimizer.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 2: 100%|██████████| 431/431 [06:07<00:00,  1.17it/s, loss=0.0333]


In [ ]:
def save_model(model, path):
  torch.save(model.state_dict(), path)

save_model(model, './roberta_ranker.pth')

In [ ]:
model.eval()
model_predicted = []
all_logits = []
with torch.no_grad():
   loop = tqdm.tqdm(test_dataloader)
   for batch in loop:
        input_ids, attention_mask, labels = (tensor.to(device) for tensor in batch)
        logits = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels.unsqueeze(1).float())
        numpy_logits = logits.cpu().detach().numpy()
        
        model_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

100%|██████████| 27/27 [00:30<00:00,  1.12s/it]


In [ ]:
print(classification_report(test_data['label'].tolist(), model_predicted))

              precision    recall  f1-score   support

           0       0.52      0.52      0.52       886
           1       0.49      0.48      0.49       838

    accuracy                           0.50      1724
   macro avg       0.50      0.50      0.50      1724
weighted avg       0.50      0.50      0.50      1724

